In [22]:
import pandas as pd
import numpy as np
import psycopg2

Data extractrion

In [23]:
equipment_data = pd.read_csv('/content/equipment_sensor.csv')
network_data = pd.read_csv('/content/maintenance_records.csv')
maintenance_data = pd.read_csv('/content/network_sensor.csv')

Data transformation 

In [24]:
# Remove duplicates from equipment data
equipment_data = equipment_data.drop_duplicates()

# Fix missing data in equipment data
equipment_data = equipment_data.fillna(method='ffill')

# Normalize the sensor reading in equipment data
equipment_data['sensor_reading'] = (equipment_data['sensor_reading'] - equipment_data['sensor_reading'].mean()) / equipment_data['sensor_reading'].std()

In [25]:
# Aggregate the equipment data to get the total number of equipment failures
equipment_failures = equipment_data.groupby(['ID']).size().reset_index(name='total_failures')

# Aggregate the equipment data to get the average time between failures
equipment_data['datetime'] = pd.to_datetime(equipment_data['date'] + ' ' + equipment_data['time'])
equipment_data['time_between_failures'] = equipment_data.groupby(['ID'])['datetime'].diff()
equipment_data['time_between_failures'] = equipment_data['time_between_failures'].dt.total_seconds() / 3600
equipment_avg_time_between_failures = equipment_data.groupby(['ID'])['time_between_failures'].mean().reset_index(name='avg_time_between_failures')

In [53]:
# Join the datasets
df_joined = pd.concat([equipment_data, network_data, maintenance_data], axis=0)


In [37]:
#Data loading to postgreSQL database 
!curl ipecho.net/plain

35.204.158.204

In [32]:
pip install psycopg2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [39]:
import psycopg2

In [54]:
conn = psycopg2.connect(
    host="34.176.59.243",
    database="post",
    user="postgresqlproject1",
    password="123456"
)

In [ ]:
# create a cursor object
cur = conn.cursor()
df_joined.to_sql(name="equipment_sensor,maintenance_records,network_sensor", con=conn, if_exists="append", index=False)
cur.close()
conn.close()